In [1]:
import pandas as pd
from datetime import datetime
import sys
import os
import matplotlib.pyplot as plt
import json
import logging
from MSSQLDB_Datamanager import DataManager

In [ ]:
## logger 設定
now = datetime.now()
log_filename = 'DBA_finproject_{}.log'.format(now.strftime('%Y-%m-%d'))
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
log_dir ='./var/log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
file_handler = logging.FileHandler('./var/log/' + log_filename)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

## config 讀取
cfg_path = r".\config.json"
with open(cfg_path, 'r', encoding='utf-8-sig') as f:
    cfg = json.load(f)
# databse connection information
db_cfg = cfg['db_connect']
db_cfg['creator'] = __import__(db_cfg['creator'])

In [ ]:
## 資料區間設定
time_start = '2022-12-01'
time_end = '2023-03-31'
## query 資料

DM = DataManager(logger, db_cfg ,time_start, time_end)
Behavior,MemberData,OrderData,OrderSlave,SalePageData,SegmentData = DM.read_data_from_db()

In [ ]:
from MSSQLDB_manager import Manager
time_start = '2022-12-01'
time_end = '2023-03-31'
db = Manager(logger, db_cfg ,time_start ,time_end)
db.get_db_table('Behavior',creat_dt = True)

In [ ]:
Behavior.info()

In [ ]:
cstm_behavoir = Behavior.groupby('FullvisitorId')
visitor_Ids = Behavior.FullvisitorId.unique().tolist()

In [ ]:
tmp = cstm_behavoir.get_group(visitor_Ids[0])

In [18]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
from Customer import *
import math
from tqdm import tqdm
import warnings
from datetime import timedelta
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv(r"BehaviorData_2023-02-01.csv")

In [5]:
df['HitDateTime'] = df['HitTime'].apply(lambda x : pd.to_datetime(x,unit='ms')+timedelta(hours=8))

In [6]:
df['EventDateTime'] = df['EventTime'].apply(lambda x : pd.to_datetime(x,unit='ms')+timedelta(hours=8))
df = df.drop(columns=['HitTime','EventTime'])


In [7]:
df.to_csv('date_transform_BehaviorData_2023-02-01.csv', index = False)

In [8]:
df.sort_values(by = ['ShopMemberId','FullvisitorId', 'HitDateTime', 'EventDateTime'], inplace= True)

In [9]:
df_copy = df.copy()

In [10]:
df_copy.drop(df_copy[df_copy.ShopMemberId.isnull()].index, inplace=True)

In [11]:
session_data = df_copy.groupby(['ShopMemberId', 'HitDateTime'])

In [12]:
ids = df.ShopMemberId.unique().tolist()

In [14]:
dciu_table = df_copy[['ShopMemberId', 'HitDateTime']]

In [15]:
dciu_table.drop_duplicates(inplace=True)

In [16]:
from datetime import datetime

In [34]:
dciu_table.sort_values(by = ['HitDateTime'], inplace = True)
dciu_table.reset_index(drop = True, inplace = True)
dciu_table.to_csv(f'dciu_{datetime.now().strftime("%Y%m%d")}')

In [35]:
dciu_table

,ShopMemberId,HitDateTime
0,BM6BBq0B4v+9mZ8q/reGWrxNIS+rVhloX8Qdvxtab5g=,2023-02-01 08:00:12.946
1,CMUcNcCnQR0Ef1sblo72oYCCJjbWPwtNA9/CFi9Y6GY=,2023-02-01 08:01:07.506
2,yfG9iHYdWWdSk/RW/k6VdOjW2Qy8tpeuJ/0ZmVSWWA0=,2023-02-01 08:01:15.172
3,cfht4mjSXC786WjpNkZahrdjUljO7QsCSsGaGiiX8sk=,2023-02-01 08:02:34.187
4,GsBR8N+I11O53IY9JZWhajo0hK7W8+A1HBR7J+LHcqs=,2023-02-01 08:02:55.009
...,...,...
294703,je7FOhqH374oEQLfcjZwKjm9Pzrzw2EbIsZy7FuWgt0=,2023-03-01 07:57:59.920
294704,dxhYyKNFK0Sh/9IYmK7EWj+2REdXBbyEo7Zm5R2AyLw=,2023-03-01 07:58:37.574
294705,2e7KEcbBoprEEUc2EYTaj77oswdsC04GVvyeUNNV0uc=,2023-03-01 07:59:00.278
294706,j4z50WByupJrGQHeBCpkEvVOqrXGqiy3jaKD0c+q180=,2023-03-01 07:59:04.373


In [41]:
new = df_copy[['ShopMemberId', 'HitDateTime','Behavior']].copy()
new = new.groupby(['ShopMemberId', 'HitDateTime', 'Behavior']).Behavior.count().unstack()
new = new.rename_axis(columns=None)
new.reset_index(inplace=True)

In [43]:
new.fillna(0, inplace=True)

In [48]:
new.purchase = new.purchase.apply(lambda x: 0 if x < 1.0 else 1)

In [52]:
new.columns

Index(['ShopMemberId', 'HitDateTime', 'add', 'checkout', 'purchase', 'register', 'search', 'selectcontent', 'viewarticledetail', 'viewcategory', 'viewcoupondetail', 'viewcustompage', 'viewecoupondetail', 'viewmainpage', 'viewproduct', 'viewpromotiondetail', 'viewvideodetail'], dtype='object')

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [65]:
X = new.drop(['ShopMemberId', 'HitDateTime', 'purchase'], axis = 1)
Y = new[['purchase']]

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [68]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
trian_pred = rf.predict(X_train)
y_pred = rf.predict(X_test)
print("trian accuracy_score", accuracy_score(y_train, trian_pred))
print("test accuracy_score", accuracy_score(y_test, y_pred))

trian accuracy_score 0.9981143508083085
test accuracy_score 0.9854206960514856


In [69]:
rf.feature_importances_

array([6.16245029e-02, 4.55202730e-01, 2.01395201e-02, 1.29159039e-02,
       1.02989949e-01, 6.68038078e-04, 7.68340765e-02, 3.54634203e-04,
       2.69232929e-02, 4.17902352e-02, 6.31497329e-02, 1.05323458e-01,
       3.20428908e-02, 4.10360450e-05])

In [70]:
X_train.columns

Index(['add', 'checkout', 'register', 'search', 'selectcontent', 'viewarticledetail', 'viewcategory', 'viewcoupondetail', 'viewcustompage', 'viewecoupondetail', 'viewmainpage', 'viewproduct', 'viewpromotiondetail', 'viewvideodetail'], dtype='object')

In [76]:
weight = {}
for i in range(len(X_train.columns)):
    weight[X_train.columns[i]] = rf.feature_importances_[i]

In [77]:
new['Weight'] = new['Behavior'].apply(lambda )

{'add': 0.06162450285927115,
 'checkout': 0.45520272982657534,
 'register': 0.020139520069162392,
 'search': 0.012915903926722996,
 'selectcontent': 0.10298994889033038,
 'viewarticledetail': 0.0006680380779737929,
 'viewcategory': 0.07683407647010693,
 'viewcoupondetail': 0.0003546342026053095,
 'viewcustompage': 0.02692329291559931,
 'viewecoupondetail': 0.04179023523081222,
 'viewmainpage': 0.06314973290570475,
 'viewproduct': 0.10532345780015884,
 'viewpromotiondetail': 0.032042890779943094,
 'viewvideodetail': 4.103604503372896e-05}

In [71]:
from sklearn.linear_model import LogisticRegression


In [74]:
lr=  LogisticRegression()
lr.fit(X_train, y_train)
tp = lr.predict(X_train)
p = lr.predict(X_test)
print("trian accuracy_score", accuracy_score(y_train, tp))
print("test accuracy_score", accuracy_score(y_test, p))

trian accuracy_score 0.9828158704767445
test accuracy_score 0.9830794113987762
